In [1]:
from glob import glob
from stopwatch import Stopwatch
import inspect
from mass import trace, execute_simulation, simulate, Agent
from matplotlib import colors
from tqdm.auto import tqdm
import matplotlib
import matplotlib.pyplot as plt
import math
import time
import simpy
from functools import partial
import json
import random
import numpy as np
import pandas as pd
import shutil
import os
import uuid
from datetime import datetime
import sys

from run_tools import is_notebook
from mass_tools import time_to_int

if is_notebook():
    PARAM_ID = 295
else:
    PARAM_ID = int(sys.argv[-1])

RANDOM_SEED = 1234

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

FOLDNAME = f"sim/{PARAM_ID:08d}_"+(str(datetime.now())+" "+uuid.uuid4().hex).replace('-',
                                                                                     '').replace(' ', '_').replace(':', '').replace('.', '_')
RUN_START = time_to_int(1,8,0)



In [2]:
class AskForFavourFrame:
    def __init__(self,topic:str,buf_size:int):
        self.topic=topic
        self.buf_size=length
    def __repr__(self):
        return f"""AskForFavourFrame(
        topic={self.topic},
        buf_size={self.buf_size})"""
    def __str__(self):
        return self.__repr__()

In [3]:
class FavourConditionsFrame:
    def __init__(self,topic:str,favour:int,timestamp):
        self.topic=topic
        self.favour=favour
        self.timestamp=timestamp
    def __repr__(self):
        return f"""FavourConditionsFrame(
        topic={self.topic},
        favour={self.favour},
        timestamp={self.timestamp})"""
    def __str__(self):
        return self.__repr__()

class POWFavourConditionsFrame(FavourConditionsFrame):
    def __init__(self,topic:str,favour:int,timestamp,pow_scheme:str,pow_target:int):
        super().__init__(topic,favour,timestamp)
        self.pow_scheme=pow_scheme
        self.pow_target=pow_target
    def __repr__(self):
        return f"""{super().__repr__()}
        <-
        POWFavourConditionsFrame(
        pow_scheme={self.pow_scheme},
        pow_target={self.pow_target})"""
    
class LNFavourConditionsFrame(FavourConditionsFrame):
    def __init__(self,topic:str,favour:int,timestamp,pow_scheme:str,ln_addr,satoshis:int):
        super().__init__(topic,favour,timestamp)
        self.ln_addr=ln_addr
        self.satoshis=satoshis
    def __repr__(self):
        return f"""{super().__repr__()}
        <-
        LNFavourConditionsFrame(
        ln_addr={self.ln_addr},
        satoshis={self.satoshis})"""


In [4]:
class pubkey:str

In [5]:
class OnionRoute:
    def __init__(self):
        self._onion=[]
        
    def peel(self,priv_key):
        return self._onion.pop()
    
    def enclose(self,peer_name,pub_key):
        self._onion.append(peer_name)
        
    def buf_size(self):
        return len(self._onion)
        
    def clone(self):
        onion=OnionRoute()
        onion._onion=self._onion
        return onion
        
    def __repr__(self):
        return f"""OnionRoute({self.onion})"""
    
    def __str__(self):
        return self.__repr__()

In [15]:
class Payload:
    def __init__(self):
        self._buf=None
        
    def encrypt(self,message,pub_key):
        self._buf=message
        
    def decrypt(self):
        return self._buf
    
    def __repr__(self):
        return f"""Payload({self._buf})"""
    
    def __str__(self):
        return self.__repr__()    

In [6]:
class BroadcastFrame:
    def __init__(self,topic:str,favour:int,timestamp,
                 message,
                thank_you_secret_pubkey:pubkey,
                reply_pubkey:pubkey,
                reply_ln_addr,
                reply_price:int,
                backward_onion:OnionRoute,
                ):
        self.topic=topic
        self.favour=favour
        self.timestamp=timestamp
        self.message=message
        self.thank_you_secret_pubkey=thank_you_secret_pubkey
        self.reply_pubkey=reply_pubkey
        self.reply_ln_addr=reply_ln_addr
        self.reply_price=reply_price
        self.backward_onion=backward_onion
    def __repr__(self):
        return f"""BroadcastFrame(
        topic={self.topic},
        favour={self.favour},
        timestamp={self.timestamp},
        message={self.message},
        thank_you_secret_pubkey={self.thank_you_secret_pubkey},
        reply_pubkey={self.reply_pubkey},
        reply_ln_addr={self.reply_ln_addr},
        reply_price={self.reply_price}
        backward_onion={self.backward_onion}
        )"""
    def __str__(self):
        return self.__repr__()

In [7]:
class POWBroadcastFrame(BroadcastFrame):
    def __init__(self,topic:str,favour:int,timestamp,
                 pow_nuance:int,
                 message,
                thank_you_secret_pubkey:pubkey,
                reply_pubkey:pubkey,
                reply_ln_addr,
                reply_price:int,
                 backward_onion:OnionRoute
                ):
        super().__init__(ident,message,
                         thank_you_secret_pubkey,
                         reply_pubkey,reply_ln_addr,reply_price,backward_onion)
        self.pow_nuance=pow_nuance    
        
    def __repr__(self):
        return f"""{super().__repr__()}
        <-
        POWBroadcastFrame(pow_nuance={self.pow_nuance})"""


In [8]:
class LNBroadcastFrame(BroadcastFrame):
    def __init__(self,topic:str,favour:int,timestamp,
                 ln_utxo,
                 message,
                thank_you_secret_pubkey:pubkey,
                reply_pubkey:pubkey,
                reply_ln_addr,
                reply_price:int,
                 backward_onion:OnionRoute
                ):
        super().__init__(ident,message,
                         thank_you_secret_pubkey,
                         reply_pubkey,reply_ln_adådr,reply_price,backward_onion)
        self.ln_utxo=ln_utxo
        
    def __repr__(self):
        return f"""{super().__repr__()}
        <-
        LNBroadcastFrame(ln_utxo={self.ln_utxo})
        """


In [ ]:
class CommunicateFrame:
    def __init__(self,payload:Payload,
                 forward_onion:OnionRoute,
                 backward_onion:OnionRoute,
                ):
        self.payload=payload
        self.forward_onion=forward_onion
        self.backward_onion=backward_onion
    def __repr__(self):
        return f"""CommunicateFrame(
        payload={self.payload},
        forward_onion={self.forward_onion},
        backward_onion={self.backward_onion}
        )"""
    def __str__(self):
        return self.__repr__()    

In [ ]:
class ThankYouFrame:
    def __init__(self,secret_key:str,
                 forward_onion:OnionRoute,
                ):
        self.secret_key=secret_key
        self.forward_onion=forward_onion
    def __repr__(self):
        return f"""ThankYouFrame(
        secret_key={self.payload},
        forward_onion={self.forward_onion}
        )"""
    def __str__(self):
        return self.__repr__()        

In [9]:
class SweetGossipNode(Agent):
    def __init__(self, context_name, name):
        super().__init__(context_name, name)
        self.name = name
        self.history = []
        self._log_i = 0
        self._known_hosts = dict()
        self._asks_for = dict()
        self._priv_key = ""
        self.pub_key=""

    def log_history(self, e, what, d={}):
        self.ctx(e, lambda: self.trace(e,str(d)))
        self.history.append(d)
        self._log_i += 1
        
    def connect_to(self,other):
        self._known_hosts[other.name]=other
        other._known_hosts[self.name]=self
        
    def calc_buf_size(self,topic:str,message:str,onion:OnionRoute):
        return len(topic)+len(message)+onion.buf_size()
        
    def broadcast(self,e,topic:str,message:str,backward_onion:OnionRoute = OnionRoute()):
        aff_frame = AskForFavourFrame(topic,calc_buf_size(topic,message,backward_onion))
        for peer in self._known_hosts.values():
            if not peer.name in self._asks_for:
                self._asks_for[peer.name]=[]
            onion.enclose(self.name,self._priv_key)
            brd = POWBroadcastFrame(topic,
                                    0,0,0,
                 message,
                "ts1",
                "rpub",
                "rln",
                1,
               backward_onion)
            self._asks_for[peer.name].append(brd)
            self.new_message(e,host,add_frame)
            
    def communicate(self,e,payload:Payload,forward_onion:OnionRoute,backward_onion:OnionRoute):
        host_name = forward_onion.peel()        
        if host_name==self.name:
            self.on_communicate(payload.decrypt(),backward_onion)
        elif host_name in self._known_hosts:
            backward_onion.enclose(self.name,self._priv_key)            
            com_frame = CommunicateFrame(payload,forward_onion,backward_onion)
            self.new_message(e,self._known_hosts[host_name],com_frame)

    def respond(self,e,message:str,forward_onion:OnionRoute):
        payload = Payload()
        payload.encrypt(message,pub_key=self.pub_key)
        communicate(e,payload,forward_onion=forward_onion,backward_onion=OnionRoute())
        
    def thankyou(self,e,secret_key:str,forward_onion:OnionRoute):
        host_name = forward_onion.peel()        
        if host_name!=self.name:
            if host_name in self._known_hosts:
                thx_frame = ThankYouFrame(secret_key,forward_onion)
                self.new_message(e,self._known_hosts[host_name],com_frame)
        on_thankyou(secret_key)
        
    
    def on_message(self, e, m):
        if isinstance(m.data,AskForFavourFrame):
            aff_frame=m.data
            peer_name=m.sender.name
            fcond=POWFavourConditionsFrame(aff_frame.topic,0,0,"SHA256",1)
            self.reply(e,m,fcond)
        elif isinstance(m.data,FavourConditionsFrame):
            fc_frame=m.data
            brd_frame=self._asks_for[peer_name].pop(0)
            if isinstance(brd_frame,POWBroadcastFrame):
                brd_frame.nuance=0
                self.reply(e,m,brd_frame)
            elif isinstance(brd_frame,LNBroadcastFrame):
                pass
        elif isinstance(m.data,BroadcastFrame):
            brd_frame=m.data
            payload=self.accept_broadcast(brd_frame)
            if payload is not None:
                self.comunicate(e,payload=payload,forward_onion=brd_frame.backward_onion)
            else:
                self.broadcast(e,topic=brd_frame.topic,message=brd_frame.message,backward_onion=brd_frame.backward_onion)
        elif isinstance(m.data,CommunicateFrame):
            com_frame=m.data
            self.communicate(e,payload=com_frame.payload,onion=com_frame.onion)
        elif isinstance(m.data,ThankYouFrame):
            thx_frame=m.data
            self.thankyou(e,secret_key=thx_frame.secret_key,onion=thx_frame.onion)
        else:
            self.ctx(e, lambda: self.trace(e, "unknown request:", m))
    
    def accept_broadcast(self,brd_frame:BroadcastFrame) -> Payload:
        return None
    
    def on_communicate(self,message:str,forward_onion:OnionRoute):
        pass
    
    def on_thankyou(self,secret_key:str):
        pass
    

In [10]:
class Gossiper(SweetGossipNode):
    pass

In [11]:
class GigWorker(SweetGossipNode):
    pass

In [12]:
class Customer(SweetGossipNode):
    
    def homeostasis(self, e):
        self.ctx(e, lambda: self.trace(e, "is starting..."))

        self.schedule(partial(self.run_job, {"run"}),
                      partial(self.on_return, e), RUN_START)
        
        yield simpy.events.AllOf(e, [e.process(self.run_scheduler(e))])

        yield e.timeout(float('inf'))
        
    def run_job(self, what, e):
        def processor():
            if False:
                yield e.timeout(0)
            self.broadcast(e,"test topic","test message")
            return None,

        self.ctx(e, lambda: self.trace(e, what))
        self.log_history(e, "run_job", what)
        return e.process(processor())
    
    def on_return(self, e, val):
        self.ctx(e, lambda: self.trace(e, val))

In [13]:
def main(sim_id):
    start = time.time()

    def printMessages(msgs):
        for m in msgs:
            print(m)

    things = {}
    
    things["Gossiper1"]=Gossiper("Gossipers","Gossiper1")
    things["GigWorker1"]=GigWorker("GigWorkers","GigWorker1")
    things["Customer1"]=Customer("Customers","Customer1")
    
    things["GigWorker1"].connect_to(things["Gossiper1"])
    things["Customer1"].connect_to(things["Gossiper1"])
        
    simulate(sim_id, things, verbose={
                        "message flow",
        "GigWorkers",
        "Gossipers",
        "Customers",
        "unknown message"}, until=float('inf'))

    for a in things:
        if(len(things[a].queue.items) > 0):
            print(a)
            printMessages(things[a].queue.items)

    end = time.time()
    print(end - start)


In [14]:
execute_simulation(main, sim_id=FOLDNAME)

sim/00000295_20221217_103240_454247_207352ca1f1d42fbb8460030b7651ebf :  ) Sun 0 00:00  | Gossiper1 STARTS
sim/00000295_20221217_103240_454247_207352ca1f1d42fbb8460030b7651ebf :  ) Sun 0 00:00  | GigWorker1 STARTS
sim/00000295_20221217_103240_454247_207352ca1f1d42fbb8460030b7651ebf :  ) Sun 0 00:00  | Customer1 is starting...
sim/00000295_20221217_103240_454247_207352ca1f1d42fbb8460030b7651ebf :  ) Mon 1 08:00  | Customer1 {'run'}
sim/00000295_20221217_103240_454247_207352ca1f1d42fbb8460030b7651ebf :  ) Mon 1 08:00  | Customer1 {'run'}


TypeError: SweetGossipNode.broadcast() missing 1 required positional argument: 'onion'